# Interpolation Explorer

Compare interpolation methods and query specific coordinates.

In [1]:
import json
import glob
from pathlib import Path
from datetime import date
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay, ConvexHull
from scipy.interpolate import LinearNDInterpolator
from scipy.spatial.distance import cdist
from sklearn.neighbors import KNeighborsRegressor
import ipywidgets as widgets
from IPython.display import display, clear_output

print('Imports OK')

Imports OK


In [2]:
import pickle
from dataclasses import dataclass

@dataclass
class User:
    name: str
    age: int
    email: str

# Create custom objects
users = [
    User("Alice", 30, "alice@example.com"),
    User("Bob", 25, "bob@example.com")
]

# Save custom objects
with open('users.pkl', 'wb') as file:
    pickle.dump(users, file, protocol=pickle.HIGHEST_PROTOCOL)
users = []
print(f"Saved {len(users)} user objects")

Saved 0 user objects


In [3]:
import pickle

# Load custom objects
with open('users.pkl', 'rb') as file:
    users = pickle.load(file)

print('Retrieved users:')
for user in users:
    print(f"- {user.name} ({user.age}): {user.email}")

Retrieved users:
- Alice (30): alice@example.com
- Bob (25): bob@example.com


In [4]:
import time
import pickle
import json
import msgpack

def benchmark_serialization(data, iterations=1000):
    """Benchmark different serialization methods"""
    
    # Pickle
    start_time = time.time()
    for _ in range(iterations):
        pickle.dumps(data, protocol=pickle.HIGHEST_PROTOCOL)
    pickle_time = time.time() - start_time
    
    # JSON
    start_time = time.time()
    for _ in range(iterations):
        json.dumps(data)
    json_time = time.time() - start_time
    
    # MessagePack
    start_time = time.time()
    for _ in range(iterations):
        msgpack.packb(data)
    msgpack_time = time.time() - start_time
    
    return {
        'pickle': pickle_time,
        'json': json_time,
        'msgpack': msgpack_time
    }

# Test with sample data
test_data = {
    'numbers': list(range(1000)),
    'strings': [f'string_{i}' for i in range(100)],
    'nested': {'level1': {'level2': {'level3': 'value'}}}
}

results = benchmark_serialization(test_data)
for method, time_taken in results.items():
    print(f"{method.capitalize()}: {time_taken:.4f} seconds")

Pickle: 0.0128 seconds
Json: 0.0494 seconds
Msgpack: 0.4265 seconds


## Configuration

In [5]:
PROJECT_ROOT = Path.cwd()
AEMET_DIR = PROJECT_ROOT / 'aemet/2022'
OUTPUT_DIR = PROJECT_ROOT / 'results'
OUTPUT_DIR.mkdir(exist_ok=True)

BBOX = {'lat_min': 38.5, 'lat_max': 40.5, 'lon_min': 1.0, 'lon_max': 4.5}

ISLANDS = {
    'Mallorca': {'lon_min': 2.3, 'lon_max': 3.5, 'lat_min': 39.25, 'lat_max': 39.95},
    'Menorca': {'lon_min': 3.8, 'lon_max': 4.35, 'lat_min': 39.8, 'lat_max': 40.1},
    'Ibiza': {'lon_min': 1.15, 'lon_max': 1.65, 'lat_min': 38.85, 'lat_max': 39.15},
}

AEMET_VARS = {
    'ta': 'Temperature (C)',
    'hr': 'Humidity (%)',
    'prec': 'Precipitation (mm)',
    'vv': 'Wind Speed (m/s)',
    'dv': 'Wind Direction (deg)',
    'pres': 'Pressure (hPa)',
}

BEACHES = {
    'Es Trenc': (39.35, 2.98),
    'Platja de Palma': (39.515, 2.74),
    'Cala Millor': (39.605, 3.385),
    'Port de Pollenca': (39.905, 3.08),
    'Cala Galdana': (39.935, 3.96),
    'Punta Prima': (39.815, 4.28),
    'Platja den Bossa': (38.88, 1.405),
    'Cala Conta': (38.97, 1.21),
}

## Load AEMET Data

In [6]:
json_files = sorted(glob.glob(str(AEMET_DIR / '*.json')))
print(f'Found {len(json_files)} JSON files')

all_records = []
for f in json_files:
    try:
        with open(f, 'r', encoding='utf-8') as file:
            data = json.loads(file.read())
            if 'datos' in data:
                all_records.extend(data['datos'])
    except:
        pass

df = pd.DataFrame(all_records)
df['fint'] = pd.to_datetime(df['fint'])

df_bal = df[
    (df['lat'] >= BBOX['lat_min']) & (df['lat'] <= BBOX['lat_max']) &
    (df['lon'] >= BBOX['lon_min']) & (df['lon'] <= BBOX['lon_max'])
].copy()

print(f'Total records: {len(df_bal):,}')
print(f'Stations: {df_bal["idema"].nunique()}')
print(f'Date range: {df_bal["fint"].min()} to {df_bal["fint"].max()}')

Found 207 JSON files
Total records: 191,296
Stations: 43
Date range: 2022-07-04 23:00:00 to 2023-01-24 22:00:00


## Interpolation Functions

In [7]:
class HullMultipointInterpolator:
    def __init__(self, n_boundary_samples=50, k_neighbors=10, power=2):
        self.n_boundary_samples = n_boundary_samples
        self.k_neighbors = k_neighbors
        self.power = power
        
    def _sample_hull_boundary(self, points, n_samples):
        hull = ConvexHull(points)
        hull_vertices = points[hull.vertices]
        hull_closed = np.vstack([hull_vertices, hull_vertices[0]])
        
        edges = np.diff(hull_closed, axis=0)
        edge_lengths = np.linalg.norm(edges, axis=1)
        total_length = edge_lengths.sum()
        
        sample_distances = np.linspace(0, total_length, n_samples, endpoint=False)
        
        boundary_pts, cumulative, edge_idx = [], 0, 0
        for d in sample_distances:
            while edge_idx < len(edge_lengths) - 1 and cumulative + edge_lengths[edge_idx] < d:
                cumulative += edge_lengths[edge_idx]
                edge_idx += 1
            t = (d - cumulative) / edge_lengths[edge_idx] if edge_lengths[edge_idx] > 0 else 0
            boundary_pts.append(hull_closed[edge_idx] + np.clip(t, 0, 1) * edges[edge_idx])
        
        return np.array(boundary_pts), hull_closed
        
    def fit(self, points, values):
        self.points = np.asarray(points)
        self.values = np.asarray(values)
        self.mean_value = np.mean(values)
        
        if len(points) < 3:
            return self
        
        self.linear = LinearNDInterpolator(self.points, self.values)
        self.delaunay = Delaunay(self.points)
        self.boundary_pts, self.hull_closed = self._sample_hull_boundary(self.points, self.n_boundary_samples)
        self.boundary_vals = self.linear(self.boundary_pts)
        
        valid = ~np.isnan(self.boundary_vals)
        self.boundary_pts = self.boundary_pts[valid]
        self.boundary_vals = self.boundary_vals[valid]
        
        self.knn = KNeighborsRegressor(
            n_neighbors=min(self.k_neighbors, len(self.boundary_pts)),
            weights='distance', p=self.power
        )
        self.knn.fit(self.boundary_pts, self.boundary_vals)
        return self
    
    def predict(self, targets):
        targets = np.asarray(targets)
        if targets.ndim == 1:
            targets = targets.reshape(1, -1)
        
        if len(self.points) < 3:
            return np.full(len(targets), self.mean_value)
        
        result = np.zeros(len(targets))
        inside_mask = self.delaunay.find_simplex(targets) >= 0
        
        if inside_mask.any():
            result[inside_mask] = self.linear(targets[inside_mask])
        if (~inside_mask).any():
            result[~inside_mask] = self.knn.predict(targets[~inside_mask])
        
        return result


def interpolate_with_method(points, values, grid_pts, method):
    n_stations = len(points)
    
    if n_stations == 0:
        return np.full(len(grid_pts), np.nan), 'No stations'
    
    if n_stations < 3:
        mean_val = np.mean(values)
        return np.full(len(grid_pts), mean_val), f'Mean ({n_stations} stations)'
    
    if method == 'hull_multipoint':
        try:
            interp = HullMultipointInterpolator(n_boundary_samples=50, k_neighbors=10)
            interp.fit(points, values)
            return interp.predict(grid_pts), 'Hull Multipoint'
        except:
            distances = cdist(grid_pts, points)
            distances = np.maximum(distances, 1e-10)
            weights = 1 / distances**2
            weights_norm = weights / weights.sum(axis=1, keepdims=True)
            return (weights_norm * values).sum(axis=1), 'IDW (fallback)'
    
    elif method == 'idw2':
        distances = cdist(grid_pts, points)
        distances = np.maximum(distances, 1e-10)
        weights = 1 / distances**2
        weights_norm = weights / weights.sum(axis=1, keepdims=True)
        return (weights_norm * values).sum(axis=1), 'IDW (power=2)'
    
    elif method == 'knn5':
        k = min(5, len(points))
        knn = KNeighborsRegressor(n_neighbors=k, weights='distance')
        knn.fit(points, values)
        return knn.predict(grid_pts), 'KNN (k=5)'
    
    return np.full(len(grid_pts), np.mean(values)), 'Mean'


def get_value_at_point(lat, lon, sel_date, sel_var, method='hull_multipoint'):
    """Get interpolated value at a specific coordinate."""
    day_data = df_bal[df_bal['fint'].dt.date == sel_date]
    station_data = day_data.groupby(['idema', 'lat', 'lon']).agg({sel_var: 'mean'}).reset_index().dropna()
    
    if len(station_data) == 0:
        return None, 'No data', 0
    
    points = station_data[['lon', 'lat']].values
    values = station_data[sel_var].values
    target = np.array([[lon, lat]])
    
    result, method_label = interpolate_with_method(points, values, target, method)
    return result[0], method_label, len(station_data)


print('Functions loaded')

Functions loaded


## 1. Query Single Coordinate

In [8]:
available_dates = sorted(df_bal['fint'].dt.date.unique())
available_vars = [v for v in AEMET_VARS.keys() if v in df_bal.columns]
beach_options = ['Custom'] + list(BEACHES.keys())

# Widgets
q_date = widgets.Dropdown(options=available_dates, value=available_dates[len(available_dates)//2], description='Date:')
q_var = widgets.Dropdown(options=[(AEMET_VARS[v], v) for v in available_vars], value='ta', description='Variable:')
q_beach = widgets.Dropdown(options=beach_options, value='Es Trenc', description='Beach:')
q_lat = widgets.FloatText(value=39.35, description='Latitude:', step=0.01)
q_lon = widgets.FloatText(value=2.98, description='Longitude:', step=0.01)
q_method = widgets.Dropdown(options=[('Hull Multipoint', 'hull_multipoint'), ('IDW', 'idw2'), ('KNN', 'knn5')], value='hull_multipoint', description='Method:')
q_btn = widgets.Button(description='Get Value', button_style='primary')
q_all_btn = widgets.Button(description='Get All Variables', button_style='info')
q_output = widgets.Output()

def on_beach_change(change):
    if change['new'] != 'Custom':
        lat, lon = BEACHES[change['new']]
        q_lat.value = lat
        q_lon.value = lon

q_beach.observe(on_beach_change, names='value')

def query_value(b):
    with q_output:
        clear_output(wait=True)
        val, method_label, n_stations = get_value_at_point(
            q_lat.value, q_lon.value, q_date.value, q_var.value, q_method.value
        )
        
        print(f'Coordinate: ({q_lat.value}, {q_lon.value})')
        print(f'Date: {q_date.value}')
        print(f'Stations used: {n_stations}')
        print(f'Method: {method_label}')
        print(f'─' * 40)
        if val is not None:
            print(f'{AEMET_VARS[q_var.value]}: {val:.2f}')
        else:
            print('No data available')

def query_all_vars(b):
    with q_output:
        clear_output(wait=True)
        print(f'Coordinate: ({q_lat.value}, {q_lon.value})')
        print(f'Date: {q_date.value}')
        print(f'Method: {q_method.value}')
        print(f'─' * 40)
        
        for var in available_vars:
            val, method_label, n_stations = get_value_at_point(
                q_lat.value, q_lon.value, q_date.value, var, q_method.value
            )
            if val is not None:
                print(f'{AEMET_VARS[var]}: {val:.2f}')
            else:
                print(f'{AEMET_VARS[var]}: N/A')
        
        print(f'─' * 40)
        print(f'Stations: {n_stations} | Method: {method_label}')

q_btn.on_click(query_value)
q_all_btn.on_click(query_all_vars)

print('QUERY WEATHER AT COORDINATE')
print('=' * 50)
display(widgets.VBox([
    widgets.HBox([q_date, q_var]),
    widgets.HBox([q_beach, q_lat, q_lon]),
    widgets.HBox([q_method, q_btn, q_all_btn]),
    q_output
]))

QUERY WEATHER AT COORDINATE


## 2. Query Multiple Beaches

In [9]:
m_date = widgets.Dropdown(options=available_dates, value=available_dates[len(available_dates)//2], description='Date:')
m_var = widgets.Dropdown(options=[(AEMET_VARS[v], v) for v in available_vars], value='ta', description='Variable:')
m_btn = widgets.Button(description='Query All Beaches', button_style='primary')
m_output = widgets.Output()

def query_all_beaches(b):
    with m_output:
        clear_output(wait=True)
        
        print(f'Date: {m_date.value} | Variable: {AEMET_VARS[m_var.value]}')
        print(f'─' * 60)
        print(f'{"Beach":<20} {"Lat":>8} {"Lon":>8} {"Value":>10} {"Method":<20}')
        print(f'─' * 60)
        
        results = []
        for name, (lat, lon) in BEACHES.items():
            val, method_label, n_stations = get_value_at_point(
                lat, lon, m_date.value, m_var.value, 'hull_multipoint'
            )
            val_str = f'{val:.2f}' if val is not None else 'N/A'
            print(f'{name:<20} {lat:>8.3f} {lon:>8.3f} {val_str:>10} {method_label:<20}')
            if val is not None:
                results.append({'beach': name, 'lat': lat, 'lon': lon, 'value': val})
        
        print(f'─' * 60)
        if results:
            vals = [r['value'] for r in results]
            print(f'Min: {min(vals):.2f} | Max: {max(vals):.2f} | Mean: {np.mean(vals):.2f}')

m_btn.on_click(query_all_beaches)

print('QUERY ALL BEACHES')
print('=' * 50)
display(widgets.VBox([
    widgets.HBox([m_date, m_var, m_btn]),
    m_output
]))

QUERY ALL BEACHES


## 3. Visualize Interpolation Map

In [10]:
island_options = ['All'] + list(ISLANDS.keys())
method_options = [('Hull Multipoint', 'hull_multipoint'), ('IDW', 'idw2'), ('KNN', 'knn5')]

v_date = widgets.Dropdown(options=available_dates, value=available_dates[len(available_dates)//2], description='Date:')
v_var = widgets.Dropdown(options=[(AEMET_VARS[v], v) for v in available_vars], value='ta', description='Variable:')
v_island = widgets.Dropdown(options=island_options, value='Mallorca', description='Island:')
v_method = widgets.Dropdown(options=method_options, value='hull_multipoint', description='Method:')
v_show_hull = widgets.Checkbox(value=True, description='Show Hull')
v_show_labels = widgets.Checkbox(value=True, description='Show Labels')
v_show_target = widgets.Checkbox(value=True, description='Show Target Point')
v_target_lat = widgets.FloatText(value=39.35, description='Target Lat:', step=0.01)
v_target_lon = widgets.FloatText(value=2.98, description='Target Lon:', step=0.01)
v_btn = widgets.Button(description='Show Map', button_style='primary')
v_output = widgets.Output()

def show_map(b):
    with v_output:
        clear_output(wait=True)
        
        sel_date = v_date.value
        sel_var = v_var.value
        sel_island = v_island.value if v_island.value != 'All' else None
        sel_method = v_method.value
        
        day_data = df_bal[df_bal['fint'].dt.date == sel_date]
        station_data = day_data.groupby(['idema', 'lat', 'lon']).agg({sel_var: 'mean'}).reset_index().dropna()
        
        if sel_island:
            bbox = ISLANDS[sel_island]
            station_data = station_data[
                (station_data['lon'] >= bbox['lon_min'] - 0.2) & 
                (station_data['lon'] <= bbox['lon_max'] + 0.2) &
                (station_data['lat'] >= bbox['lat_min'] - 0.2) & 
                (station_data['lat'] <= bbox['lat_max'] + 0.2)
            ]
        
        if len(station_data) == 0:
            print('No stations with data')
            return
        
        points = station_data[['lon', 'lat']].values
        values = station_data[sel_var].values
        
        bbox = ISLANDS[sel_island] if sel_island else BBOX
        grid_lon = np.linspace(bbox['lon_min'], bbox['lon_max'], 60)
        grid_lat = np.linspace(bbox['lat_min'], bbox['lat_max'], 60)
        grid_lon_2d, grid_lat_2d = np.meshgrid(grid_lon, grid_lat)
        grid_pts = np.column_stack([grid_lon_2d.ravel(), grid_lat_2d.ravel()])
        
        grid_values, method_label = interpolate_with_method(points, values, grid_pts, sel_method)
        grid_values = grid_values.reshape(grid_lon_2d.shape)
        
        # Get value at target point
        target_val, _, _ = get_value_at_point(
            v_target_lat.value, v_target_lon.value, sel_date, sel_var, sel_method
        )
        
        fig, ax = plt.subplots(figsize=(10, 8))
        im = ax.contourf(grid_lon_2d, grid_lat_2d, grid_values, levels=20, cmap='RdYlBu_r', extend='both')
        ax.scatter(points[:, 0], points[:, 1], c='black', s=80, marker='^', edgecolors='white', linewidth=1.5, zorder=5, label='Stations')
        
        if v_show_hull.value and len(points) >= 3:
            try:
                hull = ConvexHull(points)
                hull_pts = points[hull.vertices]
                hull_closed = np.vstack([hull_pts, hull_pts[0]])
                ax.plot(hull_closed[:, 0], hull_closed[:, 1], 'k--', linewidth=2, alpha=0.7, label='Hull')
            except:
                pass
        
        if v_show_target.value:
            ax.scatter(v_target_lon.value, v_target_lat.value, c='lime', s=200, marker='*', 
                      edgecolors='black', linewidth=2, zorder=10, label=f'Target ({target_val:.2f})')
        
        if v_show_labels.value:
            for _, row in station_data.iterrows():
                ax.annotate(f'{row[sel_var]:.1f}', (row['lon'], row['lat']), 
                           fontsize=8, ha='center', va='bottom', xytext=(0, 5), 
                           textcoords='offset points',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
        
        plt.colorbar(im, ax=ax, label=AEMET_VARS.get(sel_var, sel_var))
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        ax.set_title(f'{AEMET_VARS.get(sel_var, sel_var)} - {sel_date}\n{sel_island or "All"} | {method_label} | {len(station_data)} stations', fontweight='bold')
        ax.legend(loc='upper right')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print(f'Target ({v_target_lat.value}, {v_target_lon.value}): {target_val:.2f}')
        print(f'Station values: min={values.min():.1f}, max={values.max():.1f}, mean={values.mean():.1f}')

v_btn.on_click(show_map)

print('INTERPOLATION MAP')
print('=' * 50)
display(widgets.VBox([
    widgets.HBox([v_date, v_var, v_island]),
    widgets.HBox([v_method, v_show_hull, v_show_labels, v_show_target]),
    widgets.HBox([v_target_lat, v_target_lon, v_btn]),
    v_output
]))

INTERPOLATION MAP


## 4. Direct Function Calls

In [11]:
# Example: Get temperature at Es Trenc
val, method, n = get_value_at_point(
    lat=39.35, 
    lon=2.98, 
    sel_date=date(2022, 10, 26), 
    sel_var='ta',
    method='hull_multipoint'
)

print(f'Temperature at Es Trenc: {val:.2f} C')
print(f'Method: {method}')
print(f'Stations: {n}')

Temperature at Es Trenc: 20.89 C
Method: Hull Multipoint
Stations: 39


In [12]:
# Example: Get all variables for a coordinate
target_date = date(2022, 10, 26)
lat, lon = 39.35, 2.98

print(f'Weather at ({lat}, {lon}) on {target_date}')
print('=' * 40)

for var in available_vars:
    val, method, n = get_value_at_point(lat, lon, target_date, var)
    if val is not None:
        print(f'{AEMET_VARS[var]}: {val:.2f}')

Weather at (39.35, 2.98) on 2022-10-26
Temperature (C): 20.89
Humidity (%): 80.38
Precipitation (mm): 0.00
Wind Speed (m/s): 1.32
Wind Direction (deg): 141.98
Pressure (hPa): 1016.36


## 5. Interactive Folium Map

In [13]:
import folium
import branca.colormap as cm

def create_folium_map(sel_date, sel_var, sel_island=None, target_lat=None, target_lon=None, method='hull_multipoint'):
    day_data = df_bal[df_bal['fint'].dt.date == sel_date]
    station_data = day_data.groupby(['idema', 'lat', 'lon']).agg({sel_var: 'mean'}).reset_index().dropna()
    
    if sel_island and sel_island != 'All':
        bbox = ISLANDS[sel_island]
        station_data = station_data[
            (station_data['lon'] >= bbox['lon_min'] - 0.2) & 
            (station_data['lon'] <= bbox['lon_max'] + 0.2) &
            (station_data['lat'] >= bbox['lat_min'] - 0.2) & 
            (station_data['lat'] <= bbox['lat_max'] + 0.2)
        ]
        center = [(bbox['lat_min'] + bbox['lat_max'])/2, (bbox['lon_min'] + bbox['lon_max'])/2]
        zoom = 9
    else:
        center = [39.5, 2.9]
        zoom = 8
    
    if len(station_data) == 0:
        print('No data')
        return None
    
    points = station_data[['lon', 'lat']].values
    values = station_data[sel_var].values
    
    m = folium.Map(location=center, zoom_start=zoom, tiles=None)
    
    folium.TileLayer(
        'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri', name='Satellite'
    ).add_to(m)
    folium.TileLayer(
        'https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
        attr='Esri', name='Topographic'
    ).add_to(m)
    folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m)
    
    vmin, vmax = values.min(), values.max()
    colormap = cm.LinearColormap(['blue', 'cyan', 'yellow', 'orange', 'red'], vmin=vmin, vmax=vmax)
    colormap.caption = f'{AEMET_VARS.get(sel_var, sel_var)} - {sel_date}'
    colormap.add_to(m)
    
    # Stations
    station_group = folium.FeatureGroup(name='Stations')
    for _, row in station_data.iterrows():
        val = row[sel_var]
        color = colormap(val)
        popup = f"<b>Station</b><br>Lat: {row['lat']:.3f}<br>Lon: {row['lon']:.3f}<br><b>{AEMET_VARS.get(sel_var, sel_var)}: {val:.1f}</b>"
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=12, color='black', weight=2,
            fill=True, fillColor=color, fillOpacity=0.9,
            popup=folium.Popup(popup, max_width=200)
        ).add_to(station_group)
    station_group.add_to(m)
    
    # Convex Hull
    if len(points) >= 3:
        try:
            hull = ConvexHull(points)
            hull_pts = points[hull.vertices]
            hull_coords = [[p[1], p[0]] for p in hull_pts] + [[hull_pts[0][1], hull_pts[0][0]]]
            hull_group = folium.FeatureGroup(name='Convex Hull')
            folium.PolyLine(hull_coords, color='black', weight=3, dash_array='10', opacity=0.7).add_to(hull_group)
            hull_group.add_to(m)
        except:
            pass
    
    # Target point
    if target_lat and target_lon:
        target_val, method_label, n_stations = get_value_at_point(target_lat, target_lon, sel_date, sel_var, method)
        target_group = folium.FeatureGroup(name='Target Point')
        popup = f"<b>Target</b><br>Lat: {target_lat:.3f}<br>Lon: {target_lon:.3f}<br><b>{AEMET_VARS.get(sel_var, sel_var)}: {target_val:.2f}</b><br>Method: {method_label}"
        folium.Marker(
            location=[target_lat, target_lon],
            popup=folium.Popup(popup, max_width=250),
            icon=folium.Icon(color='green', icon='star', prefix='fa')
        ).add_to(target_group)
        target_group.add_to(m)
    
    # Beaches
    beach_group = folium.FeatureGroup(name='Beaches')
    for name, (lat, lon) in BEACHES.items():
        beach_val, _, _ = get_value_at_point(lat, lon, sel_date, sel_var, method)
        val_str = f'{beach_val:.1f}' if beach_val else 'N/A'
        popup = f"<b>{name}</b><br>{AEMET_VARS.get(sel_var, sel_var)}: {val_str}"
        folium.Marker(
            location=[lat, lon],
            popup=folium.Popup(popup, max_width=200),
            icon=folium.Icon(color='blue', icon='umbrella-beach', prefix='fa')
        ).add_to(beach_group)
    beach_group.add_to(m)
    
    folium.LayerControl().add_to(m)
    
    # Legend
    legend = f'''<div style="position:fixed;bottom:50px;left:50px;z-index:1000;background:white;padding:10px;border:2px solid #333;border-radius:5px;font-size:12px">
    <b>{AEMET_VARS.get(sel_var, sel_var)}</b><br>
    <i style="background:black;width:12px;height:12px;display:inline-block;border-radius:50%"></i> Stations ({len(station_data)})<br>
    <i style="color:green;font-size:14px">★</i> Target Point<br>
    <i style="color:blue;font-size:14px">🏖</i> Beaches<br>
    <span style="border-top:2px dashed black;width:20px;display:inline-block"></span> Hull</div>'''
    m.get_root().html.add_child(folium.Element(legend))
    
    return m

print('Folium map function loaded')

Folium map function loaded


In [14]:
f_date = widgets.Dropdown(options=available_dates, value=available_dates[len(available_dates)//2], description='Date:')
f_var = widgets.Dropdown(options=[(AEMET_VARS[v], v) for v in available_vars], value='ta', description='Variable:')
f_island = widgets.Dropdown(options=['All'] + list(ISLANDS.keys()), value='Mallorca', description='Island:')
f_beach = widgets.Dropdown(options=['None'] + list(BEACHES.keys()), value='Es Trenc', description='Target:')
f_lat = widgets.FloatText(value=39.35, description='Target Lat:', step=0.01)
f_lon = widgets.FloatText(value=2.98, description='Target Lon:', step=0.01)
f_btn = widgets.Button(description='Show Map', button_style='primary')
f_save_btn = widgets.Button(description='Save HTML', button_style='info')
f_output = widgets.Output()

def on_f_beach_change(change):
    if change['new'] != 'None':
        lat, lon = BEACHES[change['new']]
        f_lat.value = lat
        f_lon.value = lon

f_beach.observe(on_f_beach_change, names='value')

current_map = None

def show_folium_map(b):
    global current_map
    with f_output:
        clear_output(wait=True)
        current_map = create_folium_map(
            f_date.value, f_var.value, f_island.value,
            f_lat.value, f_lon.value
        )
        if current_map:
            display(current_map)

def save_map(b):
    global current_map
    if current_map:
        filename = OUTPUT_DIR / f'map_{f_var.value}_{f_date.value}.html'
        current_map.save(str(filename))
        with f_output:
            print(f'Saved: {filename}')

f_btn.on_click(show_folium_map)
f_save_btn.on_click(save_map)

print('INTERACTIVE FOLIUM MAP')
print('=' * 50)
display(widgets.VBox([
    widgets.HBox([f_date, f_var, f_island]),
    widgets.HBox([f_beach, f_lat, f_lon]),
    widgets.HBox([f_btn, f_save_btn]),
    f_output
]))

INTERACTIVE FOLIUM MAP


## 6. Quick Map (Direct Call)

In [15]:
# Quick map for Es Trenc
m = create_folium_map(
    sel_date=date(2022, 10, 26),
    sel_var='ta',
    sel_island='Mallorca',
    target_lat=39.35,
    target_lon=2.98
)
m